In [3]:
import requests
from requests import Response
from pprint import pprint
from pydantic import BaseModel, Field, ValidationError, root_validator
from typing import List

class UBike(BaseModel):
    sna: str  # 站點名稱
    sarea: str  # 站點區域
    mday: str  # 資料更新時間
    ar: str  # 站點地址
    act: bool  # 是否可用
    total: int  # 總車位數
    available_rent_bikes: int = Field(..., alias='sbi')  # 可租借車輛數
    latitude: float = Field(..., alias='lat')  # 緯度
    longitude: float  # 經度
    available_return_bikes: int = Field(..., alias='bemp')  # 可還車位數

    #@root_validator(pre=True)
    def parse_fields(cls, values):
        # 將 'act' 字段轉換為布爾值
        values['act'] = values['act'] == '1'
        return values

class UBikeRoot(BaseModel):
    root: List[UBike]  # UBike 的列表

def main():
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    res: Response = requests.get(youbike_url)
    data = res.json()

    try:
        # 解析數據
        ubike_root = UBikeRoot(root=[UBike(**item) for item in data])
        
        # 將解析後的數據轉換為字典列表
        data_list: List[dict] = ubike_root.model_dump()["root"]
        
        # 過濾出維護中的站點（act == False 的站點）
        filter_list: List[dict] = [item.dict() for item in ubike_root.root if item.act]

        # 過濾出可借車輛數為3輛以內的站點
        rent_bike_stations = [station for station in filter_list if station['available_rent_bikes'] <= 3]

        # 過濾出可還車輛數為3輛以內的站點
        return_bike_stations = [station for station in filter_list if station['available_return_bikes'] <= 3]

        print("可借車輛數為3輛以內的站點:")
        pprint(rent_bike_stations)

        print("\n可還車輛數為3輛以內的站點:")
        pprint(return_bike_stations)
        
    except ValidationError as e:
        print(e.json())

if __name__ == "__main__":
    main()


[{"type":"missing","loc":["sbi"],"msg":"Field required","input":{"sno":"500101001","sna":"YouBike2.0_捷運科技大樓站","sarea":"大安區","mday":"2024-06-14 18:54:21","ar":"復興南路二段235號前","sareaen":"Daan Dist.","snaen":"YouBike2.0_MRT Technology Bldg. Sta.","aren":"No.235， Sec. 2， Fuxing S. Rd.","act":"1","srcUpdateTime":"2024-06-14 18:56:26","updateTime":"2024-06-14 18:56:52","infoTime":"2024-06-14 18:54:21","infoDate":"2024-06-14","total":28,"available_rent_bikes":2,"latitude":25.02605,"longitude":121.5436,"available_return_bikes":26},"url":"https://errors.pydantic.dev/2.7/v/missing"},{"type":"missing","loc":["lat"],"msg":"Field required","input":{"sno":"500101001","sna":"YouBike2.0_捷運科技大樓站","sarea":"大安區","mday":"2024-06-14 18:54:21","ar":"復興南路二段235號前","sareaen":"Daan Dist.","snaen":"YouBike2.0_MRT Technology Bldg. Sta.","aren":"No.235， Sec. 2， Fuxing S. Rd.","act":"1","srcUpdateTime":"2024-06-14 18:56:26","updateTime":"2024-06-14 18:56:52","infoTime":"2024-06-14 18:54:21","infoDate":"2024-06-14","t